# Build RAPTOR RAG

*_Algorithm_*
- Input: array of strings
- Step 1: split each string into contiguous chunks with len <= 100
- Step 2: group chunks into clusters
- step 3: summarize the clusters

Ref: https://github.com/langchain-ai/langchain/blob/master/cookbook/RAPTOR.ipynb

## Lessons
- split the orginal text into chunks
- embed question into document

In [1]:
import sys
sys.path.append("../..")
from src.utils.text_utils import create_splitter

splitter = create_splitter(chunk_size=200, overlap=0, separators=[".", "\n"])

In [15]:
help(splitter.create_documents)

Help on method create_documents in module langchain.text_splitter:

create_documents(texts: 'List[str]', metadatas: 'Optional[List[dict]]' = None) -> 'List[Document]' method of langchain.text_splitter.RecursiveCharacterTextSplitter instance
    Create documents from a list of texts.



## Load data

In [2]:
from datasets import load_dataset
REPO = 'BroDeadlines/TEST.edu_tdt_proposition_data'
SPLIT = 'TEST.basic_index_TDT_clean'
repo_data = load_dataset(REPO, split=SPLIT)
repo_data

Dataset({
    features: ['content', 'url', 'doc_id', 'shards', 'splits', 'split', 'propositions', 'proposition_list'],
    num_rows: 20
})

In [7]:
from src.utils.type_utils import create_langchain_doc

docs_texts = []

def make_langchain_documents(row):
    docs = splitter.create_documents([row['content']], [{"doc_id": row['doc_id']}])
    docs_texts.extend(docs)
    return row
    
d = repo_data.map(make_langchain_documents)
len(docs_texts)

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

99

In [8]:
docs_texts[0].metadata['doc_id'] == docs_texts[1].metadata['doc_id']

True

In [9]:
docs_texts[1]

Document(page_content='. Điểm trúng tuyển  Phương thức xét tuyển theo kết quả học tập THPT (PT1) – Đợt bổ sung (xét theo điểm 6HK): Điểm xét tuyển được thực hiện theo đúng đề án tuyển sinh đại học năm 2022, thang điểm 40 và được làm tròn đến 02 chữ số thập phân (đã bao gồm điểm ưu tiên khu vực, đối tượng, hệ số trường THPT, điểm ưu tiên thành tích học sinh giỏi).  Phương thức xét tuyển theo điểm thi THPT năm 2022 (PT2) : Điểm xét tuyển được thực hiện theo đúng đề án tuyển sinh đại học năm 2022, là tổng điểm của 3 môn theo tổ hợp (có nhân hệ số môn theo tổ hợp, ngành xét tuyển theo thang điểm 40), cộng với điểm ưu tiên khu vực, đối tượng theo thang điểm 40 (nếu có), được làm tròn đến 2 chữ số thập phân theo quy định của Bộ GD&ĐT', metadata={'doc_id': 'ebe883e0-13cc-11ef-b548-0242ac1c000c'})

## Build RAPTOR

In [2]:
import random

rand_docs = random.sample(docs_texts, 20)
len(rand_docs)

NameError: name 'docs_texts' is not defined

In [11]:
from src.rag.raptor import RAPTOR
from src.service.provider import ProviderService
provider = ProviderService()
raptor = RAPTOR(provider)

vec_idx = "vec-raptor-basic_index_tdt_clean"
text_idx = "text-raptor-basic_index_tdt_clean"

es = provider.get_elasticsearch_store(vec_idx)
bm = provider.get_elasticsearch_bm25(text_idx)

In [12]:
leaf_ids = [d.metadata['doc_id'] for d in rand_docs]
leaf_texts = [d.page_content for d in rand_docs]
results = raptor.recursive_embed_cluster_summarize(leaf_texts, leaf_ids, level=1, n_levels=3)

--Generated 1 clusters--


In [16]:
results[1]

(                                                text  \
 0  .vn Đại học  Phương thức tuyển sinh 2024  Quy ...   
 1  .  Các kỹ năng sáng tác, thiết kế trong lĩnh v...   
 2  .  Các hồ sơ đủ điều kiện sẽ được Hội đồng tuy...   
 3  .vn Đại học  Phương thức tuyển sinh 2024  Quy ...   
 4  . THỜI GIAN THÔNG BÁO KẾT QUẢ XÉT TUYỂN VÀ XÁC...   
 5  .vn  Tư vấn liên kết đào tạo quốc tế:  Hotline...   
 6  Theo quy định hiện tại nếu sinh viên đã được g...   
 7  .  Với cách tiếp cận và xây dựng chương trình ...   
 8  .vn Đại học  Phương thức tuyển sinh 2024  Quy ...   
 9  .  2015: FCU lọt vào danh sách các trường đứng...   
 
                                                 embd cluster  \
 0  [0.027457751, 0.0007100709, -0.02268081, -0.01...     [0]   
 1  [0.05544808, -0.018173132, -0.025689853, -0.02...     [0]   
 2  [0.018899338, -0.013088818, -0.06511452, -0.02...     [0]   
 3  [0.027457751, 0.0007100709, -0.02268081, -0.01...     [0]   
 4  [0.023355095, -0.013827138, -0.03150953, -

In [14]:
# Initialize all_texts with leaf_texts
all_texts = leaf_texts.copy()

# Iterate through the results to extract summaries from each level and add them to all_texts
for level in sorted(results.keys()):
    # Extract summaries from the current level's DataFrame
    summaries = results[level][1]["summaries"].tolist()
    # Extend all_texts with the summaries from the current level
    all_texts.extend(summaries)



In [15]:
es.add_texts(all_texts)

['a2ff63ad-44a5-4a13-b7ed-5a93b5dd3a68',
 '6f6ffe8d-63f1-40d4-a625-5a9d329d65dc',
 '93895eab-6982-4acc-91e2-bb964de17b81',
 '93c359bd-d3c5-43f4-a27e-7d9107ffadf3',
 'e76036b0-75d6-4671-b868-ccc3c6773ca4',
 '80c63860-5900-47b4-8c32-ea972a5729d0',
 '233eaacf-e6c3-40c7-ba43-ef962747f27e',
 '4ce0cd93-3af7-47c5-b5d0-f7ad06a097e0',
 '59eac8bf-c805-4e2c-9390-08f47a09e8bf',
 '07a6194d-6010-4501-b985-e351329d79e4',
 '3bc79d4f-dcfd-4abf-b953-89529cb96c54',
 'dc255e0b-91e5-4675-b6e9-55ef880ad38e',
 'e0b8e576-6411-4b1b-ae22-0958ffa95e34',
 'd6e741a5-c2e4-4913-a132-dc960abd5672',
 '9b9e278e-0af9-4fba-a768-539dbe065f86',
 'ef461593-705d-47fa-afc0-b4e68ff7fec6',
 'a906fb31-5177-4c9b-a1a9-aaeb5dc4168d',
 '98936e67-45e3-4c3f-8b95-3c8c16af0d35',
 'f8333c91-8270-4a1d-a7bf-8c1bfe1024aa',
 'c90d52e6-c83a-45dc-8ebe-289787d114ef',
 'da25f970-f1da-4b05-b86b-a32537f4026a',
 '77faafa5-445e-47a7-8ce1-656218ca4ab9',
 '0d8d7b8d-e4f9-40eb-b175-dc108f03e31f',
 '7c4e3d3b-5b73-42ea-b404-9d411a403958',
 'd2e83ec3-42e4-

In [18]:
bm.add_texts(all_texts, metadatas={})

['920e5b64-53de-4b83-87c5-380e29d4317e',
 'b623f279-f765-4d6f-ac43-6ec429f4ca57',
 'd6e90354-7889-4246-a066-4ab13ebe920b',
 '8ea50352-2a13-4c5e-8cc8-2a2d3ff703a1',
 '9c56c2ff-ba30-464b-ba11-e6905649a2e3',
 'b15ecb6f-de3d-4957-b0cc-a9065ab1c853',
 '1cf39952-668d-49e6-95fc-39447b921a95',
 '84d30c46-1506-4302-bc88-0a268fa9f42f',
 '29ff4d79-20d5-45f7-8332-0e8d25b31bb8',
 '33524bff-27d0-416b-9362-35fe8fc96881',
 '7001c763-e340-4284-a003-fe34163a65f1',
 '44436ffb-8e4e-4551-94a5-bdab65b56d60',
 '925b6ce4-e6be-4c97-a313-9d59a1776093',
 '2248f9de-e5c0-4aa1-ad1d-d2a9e3c34ebb',
 'f1100b56-f273-4d3b-8ae0-89addeffa4e8',
 'e7922445-fc58-45e3-89ea-2d03e5e82812',
 'dc020d32-40b1-4626-98b2-4c620d8ef0dc',
 'dff9294f-91c0-47aa-8af5-21195d766604',
 '7c382646-f4ba-486f-8c38-bea42fd263b8',
 '84d6e64e-96a8-4152-8103-4c4ed932a4be',
 '250b37f9-c81f-416a-93dc-38fd25b5eabf',
 '3f83f642-7814-451e-9c83-54b6792fda5f',
 '203aa2a9-2042-4a18-8ef7-c5c7ef95ccf6',
 '6ed59172-cfc3-478e-9bca-4f1c7997ab80',
 '227a255d-5643-

In [5]:
from src.rag.hyde_rag import HydeRAG, HydeHybridSearchRAG
from src.utils.config_utils import get_gemini_hyde_config

# rag = HydeRAG(provider=provider, index="test-basic_test_tdt_dataset")
config = get_gemini_hyde_config()
config.vector_index = vec_idx
config.text_index = text_idx

# config.vector_index = "vec-sentence-propositon_medium_edu_tdt"
# config.text_index = 'text-sentence-propositon_medium_edu_tdt'

# rag = HydeRAG(provider=provider, index=config.vector_index)
rag = HydeHybridSearchRAG(provider=provider,config=config, k=4)



In [6]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
retriever = rag.ensemble_retriever
model = provider.get_simple_gemini_pro()
# Prompt
prompt = hub.pull("rlm/rag-prompt")


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Question
# rag_chain.invoke("How to define a RAG chain? Give me a specific code example.")

In [7]:
provider.config.enable_tracing("CODEBOT")

In [10]:
# Question

# ques = "Lệ phí tập sự nghề nghiệp,ôn và thi tốt nghiệp,khóa luận tính như thế nào"
ques = "Sinh viên muốn làm giấy xác nhận để đi xe buýt thì phải làm sao"
ques = "Sinh viên thi TOEIC/MOS do IIG tổ chức tại trường Đại học Tôn Đức Thắng nhưng CMND bị mờ, ép dẻo, ép lụa, ép lại, bong tróc mép để hở phần giấy bên trong thì làm thế nào"
ques = "Các lỗi vi phạm khi Sinh viên ở ký túc xá có liên quan hay ảnh hưởng gì với kết quả học tập rèn luyện gì trường học hay không"
answ = rag_chain.invoke(ques)

answ

'Tôi không tìm thấy thông tin về mối liên hệ giữa các lỗi vi phạm của sinh viên ở ký túc xá với kết quả học tập rèn luyện của họ trong tài liệu được cung cấp.'